In [1]:
# Importing required libraries
from IPython.display import display
import numpy as np
import pandas as pd
import random
import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import os
import glob

c:\users\sashah8\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

In [4]:
class GraphAttentionLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat
        
        # Initializing weights and activations
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)
        
        
    def _prepare_attentional_mechanism_input(self, Wh):
        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
        e = Wh1 + Wh2.T
        return self.leakyrelu(e)
    

    def forward(self, input, adj):
        
        # Applying linear transformation to each node
        Wh = torch.mm(input, self.W)
        
        # Performing the dot product of a_input & activations and applying leakyReLU to the dot product
        e = self._prepare_attentional_mechanism_input(Wh)
        
        # Returning a tensor filled with 1, with the same shape
        zero_vec = -9e15*torch.ones_like(e) 
        
        # If nodes not connected then put attention equal to zero
        attention = torch.where(adj > 0, e, zero_vec)
        
        # Applying softmax to the attentions
        attention = F.softmax(attention, dim=1)
        # attention = F.softmax(e, dim=1)
        
        # Applying dropout to avoid overfitting
        attention = F.dropout(attention, self.dropout, training=self.training)
        
        # Embeddings are aggregated from neighbors and scaled by the attentions
        h_prime = torch.matmul(attention, Wh)
        
        # For aggregation in intermediate layers, we apply concat and in final layer aggregation we apply average.
        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime
    

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [5]:
class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout
        self.nclass = nclass
        
        # Concatenating the outputs with all the heads from intermediate layers 
        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)
  
        self.out_att = GraphAttentionLayer(nhid * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)
        
        self.linears = torch.nn.ModuleList([torch.nn.Linear(nclass, 1) for _ in range(nclass)])
        
        

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.relu(self.out_att(x, adj))
        out = [self.linears[i](x[i, :]) for i in range(self.nclass)]

        return out

## Processing the file to input into the model

In [6]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\DOE\DOE_Q1\L1\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['MIDATL.csv', 'SOUTH.csv', 'WEST.csv']


In [7]:
# Just for previewing the column name
pd.read_csv(os.path.join(path, fileList[0]))

,Net,Temperature
0,31660.636,-7.6
1,30766.066,-8.4
2,30201.596,-8.3
3,29901.406,-6.8
4,30084.427,-7.0
...,...,...
43819,31844.076,4.8
43820,30687.078,4.4
43821,29583.295,3.7
43822,28458.691,2.8


## Processing the load data into train-test split

In [8]:
# Fetching and concatenating the data
Load_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Net"]) for fileName in fileList], axis= 1)
Load_DS.columns = [i.removesuffix('.csv') for i in fileList]
Load_DS = Load_DS.add_prefix("LOAD_")

Load_DS

,LOAD_MIDATL,LOAD_SOUTH,LOAD_WEST
0,31660.636,12571.309,49769.768
1,30766.066,12449.182,48714.914
2,30201.596,12295.476,47880.328
3,29901.406,12354.318,47380.142
4,30084.427,12542.606,47241.559
...,...,...,...
43819,31844.076,11682.100,49258.613
43820,30687.078,11442.870,47886.874
43821,29583.295,11166.770,46582.286
43822,28458.691,10838.550,45132.982


In [9]:
# Creating 1-day lag loads
Load_lag_1 = Load_DS.shift(24).fillna(0)
Load_lag_1.columns = [i.removeprefix('LOAD_') for i in Load_lag_1.columns]
Load_lag_1 = Load_lag_1.add_prefix("LAG1_")
Load_lag_1

,LAG1_MIDATL,LAG1_SOUTH,LAG1_WEST
0,0.000,0.000,0.000
1,0.000,0.000,0.000
2,0.000,0.000,0.000
3,0.000,0.000,0.000
4,0.000,0.000,0.000
...,...,...,...
43819,33159.726,10860.510,49037.954
43820,32353.072,10608.808,48438.677
43821,31132.848,10213.428,47229.754
43822,29356.335,9765.158,45404.894


In [10]:
# Creating 7-day lag loads
Load_lag_7 = Load_DS.shift(24*7).fillna(0)
Load_lag_7.columns = [i.removeprefix('LOAD_') for i in Load_lag_7.columns]
Load_lag_7 = Load_lag_7.add_prefix("LAG7_")
Load_lag_7

,LAG7_MIDATL,LAG7_SOUTH,LAG7_WEST
0,0.000,0.000,0.000
1,0.000,0.000,0.000
2,0.000,0.000,0.000
3,0.000,0.000,0.000
4,0.000,0.000,0.000
...,...,...,...
43819,31223.944,11705.914,42521.702
43820,30934.643,11709.951,42381.205
43821,30619.720,11663.864,42197.353
43822,29955.427,11519.032,41556.164


In [11]:
# Setting the train-test split
ratio = 0.2
Num_test, Num_train = int(len(Load_DS) * ratio), len(Load_DS) - int(len(Load_DS) * ratio)

In [12]:
# Scaling the load data wrt WEST
mmScaler_load = MinMaxScaler()

# Splitting the data into train and test [LOAD]
Load_train, Load_test = Load_DS[:Num_train], Load_DS[Num_train:]
print("Raw load - Train: ")
display(Load_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 1]
Load_Lag1_train, Load_Lag1_test = Load_lag_1[:Num_train], Load_lag_1[Num_train:]
print("Raw lag 1 load - Train: ")
display(Load_Lag1_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 7]
Load_Lag7_train, Load_Lag7_test = Load_lag_7[:Num_train], Load_lag_7[Num_train:]
print("Raw lag 7 load - Train: ")
display(Load_Lag7_train.head(2))
print("\n")

Raw load - Train: 


,LOAD_MIDATL,LOAD_SOUTH,LOAD_WEST
0,31660.636,12571.309,49769.768
1,30766.066,12449.182,48714.914




Raw lag 1 load - Train: 


,LAG1_MIDATL,LAG1_SOUTH,LAG1_WEST
0,0.0,0.0,0.0
1,0.0,0.0,0.0




Raw lag 7 load - Train: 


,LAG7_MIDATL,LAG7_SOUTH,LAG7_WEST
0,0.0,0.0,0.0
1,0.0,0.0,0.0


In [13]:
# Scaling the data using mix-max scaler [TRAINING]
Load_train = mmScaler_load.fit_transform(Load_train)
print("Scaled load - Train: ")
display(Load_train)

Load_Lag1_train = mmScaler_load.transform(Load_Lag1_train)
print("Scaled lag 1 load - Train: ")
display(Load_Lag1_train)

Load_Lag7_train = mmScaler_load.transform(Load_Lag7_train)
print("Scaled lag 7 load - Train: ")
display(Load_Lag7_train)

Scaled load - Train: 


array([[0.43065024, 0.46215937, 0.50711927],
       [0.41002199, 0.45465502, 0.48722235],
       [0.39700564, 0.44521024, 0.47148018],
       ...,
       [0.50850536, 0.44495941, 0.46824365],
       [0.48777324, 0.42529905, 0.46160865],
       [0.45314908, 0.39972509, 0.42831099]])

Scaled lag 1 load - Train: 


c:\users\sashah8\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- LAG1_MIDATL
- LAG1_SOUTH
- LAG1_WEST
Feature names seen at fit time, yet now missing:
- LOAD_MIDATL
- LOAD_SOUTH
- LOAD_WEST

  warnings.warn(message, FutureWarning)


array([[-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       ...,
       [ 0.45386016,  0.40100908,  0.44641797],
       [ 0.46159664,  0.41663176,  0.47836415],
       [ 0.45793298,  0.41360616,  0.47607833]])

Scaled lag 7 load - Train: 


c:\users\sashah8\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- LAG7_MIDATL
- LAG7_SOUTH
- LAG7_WEST
Feature names seen at fit time, yet now missing:
- LOAD_MIDATL
- LOAD_SOUTH
- LOAD_WEST

  warnings.warn(message, FutureWarning)


array([[-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       [-0.29942531, -0.31031077, -0.43165036],
       ...,
       [ 0.47526171,  0.43326407,  0.44862886],
       [ 0.45846524,  0.4382983 ,  0.45863275],
       [ 0.43902549,  0.43183898,  0.44878566]])

In [14]:
# Scaling the data using mix-max scaler [TESTING]
Load_test = mmScaler_load.transform(Load_test)
print("Scaled load - Test: ")
display(Load_test)

Load_Lag1_test = mmScaler_load.transform(Load_Lag1_test)
print("Scaled lag 1 load - Test: ")
display(Load_Lag1_test)

Load_Lag7_test = mmScaler_load.transform(Load_Lag7_test)
print("Scaled lag 7 load - Test: ")
display(Load_Lag7_test)

Scaled load - Test: 


array([[0.420077  , 0.3703234 , 0.39565754],
       [0.38591386, 0.33897562, 0.36205661],
       [0.35196082, 0.30448827, 0.33430867],
       ...,
       [0.38274799, 0.37585456, 0.4469962 ],
       [0.35681528, 0.3556864 , 0.41965907],
       [0.33305307, 0.34057804, 0.3888862 ]])

Scaled lag 1 load - Test: 


c:\users\sashah8\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- LAG1_MIDATL
- LAG1_SOUTH
- LAG1_WEST
Feature names seen at fit time, yet now missing:
- LOAD_MIDATL
- LOAD_SOUTH
- LOAD_WEST

  warnings.warn(message, FutureWarning)


array([[0.45516996, 0.40676053, 0.47418211],
       [0.43652896, 0.39468145, 0.45711798],
       [0.40442413, 0.37164187, 0.42807378],
       ...,
       [0.41847976, 0.31727448, 0.4592089 ],
       [0.37751442, 0.2897296 , 0.42478794],
       [0.34016058, 0.25654781, 0.38302336]])

Scaled lag 7 load - Test: 


c:\users\sashah8\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- LAG7_MIDATL
- LAG7_SOUTH
- LAG7_WEST
Feature names seen at fit time, yet now missing:
- LOAD_MIDATL
- LOAD_SOUTH
- LOAD_WEST

  warnings.warn(message, FutureWarning)


array([[0.42808262, 0.43024529, 0.44452534],
       [0.41668037, 0.42899306, 0.43979978],
       [0.39615558, 0.41911268, 0.42448822],
       ...,
       [0.40664733, 0.40639953, 0.36428651],
       [0.39132913, 0.39750003, 0.35219224],
       [0.36766393, 0.3849365 , 0.33036574]])

In [15]:
## Transformation assigns perfectly.
Load_train[0]

array([0.43065024, 0.46215937, 0.50711927])

In [16]:
Load_Lag1_train[24]

array([0.43065024, 0.46215937, 0.50711927])

In [17]:
Load_Lag7_train[24*7]

array([0.43065024, 0.46215937, 0.50711927])

In [18]:
## Transformation assigns perfectly.
Load_test[5]

array([0.26187365, 0.22331367, 0.25204169])

In [19]:
Load_Lag1_test[24+5]

array([0.26187365, 0.22331367, 0.25204169])

In [20]:
Load_Lag7_test[24*7+5]

array([0.26187365, 0.22331367, 0.25204169])

## Processing the temperature data in train test split

In [21]:
# Fetching and concatenating the data
Temp_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Temperature"]) for fileName in fileList], axis= 1)
Temp_DS.columns = [i.removesuffix('.csv') for i in fileList]
Temp_DS = Temp_DS.add_prefix("TEMP_")

Temp_DS

,TEMP_MIDATL,TEMP_SOUTH,TEMP_WEST
0,-7.6,0.5,-9.0
1,-8.4,0.1,-9.0
2,-8.3,-0.5,-9.0
3,-6.8,-0.7,-9.0
4,-7.0,-0.7,-9.1
...,...,...,...
43819,4.8,9.9,-1.3
43820,4.4,10.2,-1.5
43821,3.7,9.6,-1.7
43822,2.8,9.0,-2.1


In [22]:
# Scaling the temperature data for each individual region
mmScaler_temp = MinMaxScaler()

# Splitting the data into train and test
Temp_train, Temp_test = Temp_DS[:Num_train], Temp_DS[Num_train:]
print("Raw temperature - Train: ")
display(Temp_train.head(5))
print("Raw temperature - Test: ")
display(Temp_test.head(5))
print("\n")

# Scaling the data using mix-max scaler
Temp_train = mmScaler_temp.fit_transform(Temp_train)
print("Scaled temperature - Train: ")
display(Temp_train)

Temp_test = mmScaler_temp.transform(Temp_test)
print("Scaled temperature - Test: ")
display(Temp_test)

Raw temperature - Train: 


,TEMP_MIDATL,TEMP_SOUTH,TEMP_WEST
0,-7.6,0.5,-9.0
1,-8.4,0.1,-9.0
2,-8.3,-0.5,-9.0
3,-6.8,-0.7,-9.0
4,-7.0,-0.7,-9.1


Raw temperature - Test: 


,TEMP_MIDATL,TEMP_SOUTH,TEMP_WEST
35060,5.1,5.2,9.8
35061,5.1,5.4,10.2
35062,5.3,5.6,11.1
35063,4.8,5.9,12.1
35064,4.3,7.2,12.9




Scaled temperature - Train: 


array([[0.29043478, 0.36007828, 0.29174664],
       [0.27652174, 0.35225049, 0.29174664],
       [0.27826087, 0.34050881, 0.29174664],
       ...,
       [0.52173913, 0.44031311, 0.60268714],
       [0.51652174, 0.43052838, 0.61612284],
       [0.51478261, 0.44618395, 0.63339731]])

Scaled temperature - Test: 


array([[0.51130435, 0.45205479, 0.65259117],
       [0.51130435, 0.45596869, 0.66026871],
       [0.51478261, 0.45988258, 0.67754319],
       ...,
       [0.48695652, 0.53816047, 0.4318618 ],
       [0.47130435, 0.52641879, 0.42418426],
       [0.46608696, 0.5146771 , 0.41458733]])

# Preparing the training and testing split for model input

In [23]:
X_train = np.array([[[i, j, k] for i,j,k in zip(Temp_train[m], Load_Lag1_train[m], Load_Lag7_train[m])] for m in range(len(Temp_train))])
print(X_train.shape)
print(X_train[0])

(35060, 3, 3)
[[ 0.29043478 -0.29942531 -0.29942531]
 [ 0.36007828 -0.31031077 -0.31031077]
 [ 0.29174664 -0.43165036 -0.43165036]]


In [24]:
X_test = np.array([[[i, j, k] for i,j,k in zip(Temp_test[m], Load_Lag1_test[m], Load_Lag7_test[m])] for m in range(len(Temp_test))])
print(X_test.shape)
print(X_test[0])

(8764, 3, 3)
[[0.51130435 0.45516996 0.42808262]
 [0.45205479 0.40676053 0.43024529]
 [0.65259117 0.47418211 0.44452534]]


In [25]:
# Setting up the batch and node parameters
num_batch, num_node = Load_DS.shape

number_feat = X_train.shape[2]

print("No. of batches: ", num_batch)
print("No. of nodes: " , num_node)
print("No. of features: ", number_feat)

No. of batches:  43824
No. of nodes:  3
No. of features:  3


In [26]:
X_train, X_test = Variable(torch.FloatTensor(X_train)), torch.FloatTensor(X_test)
Y_train, Y_test = Variable(torch.FloatTensor(Load_train)), Load_test

## DEFINING THE MODEL

In [27]:
adj = np.ones((num_node, num_node))/num_node
adj = Variable(torch.FloatTensor(adj))
adj

tensor([[0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333],
        [0.3333, 0.3333, 0.3333]])

In [28]:
parser = argparse.ArgumentParser()
parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training.')
parser.add_argument('--seed', type=int, default=20, help='Random seed.')
parser.add_argument('--epochs', type=int, default=10000, help='Number of epochs to train.')
parser.add_argument('--lr', type=float, default=0.0001, help='Initial learning rate.')
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay (L2 loss on parameters).')
parser.add_argument('--hidden', type=int, default=12, help='Number of hidden units.')
parser.add_argument('--nb_heads', type=int, default=3, help='Number of head attentions.')
parser.add_argument('--dropout', type=float, default=0., help='Dropout rate (1 - keep probability).')
parser.add_argument('--alpha', type=float, default=0.2, help='Alpha for the leaky_relu.')
parser.add_argument('--batch_size', type=float, default=Num_train, help='batch size')

parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [29]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

In [30]:
model = GAT(nfeat=number_feat, 
            nhid=args.hidden, 
            nclass= num_node, #1
            dropout=args.dropout, 
            nheads=args.nb_heads,
            alpha=args.alpha)

In [31]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

In [33]:
if args.cuda:
    torch.cuda.manual_seed(args.seed)
    X_train = X_train.to(device)
    adj = adj.to(device)
    Y_train = Y_train.to(device)
    model.to(device)
    print("Yes")

In [34]:
Epochs = 150

dataset = TensorDataset(X_train, Y_train)    
loader = DataLoader(dataset, batch_size = 64, shuffle=False) 

In [35]:
model.train()

for epoch in range(Epochs):
    loss = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        if args.cuda:
            x = x.cuda()
            y = y.cuda()
            adj = adj.cuda()
        loss_train = 0
        for i in range(x.size(0)):
            output = model(x[i], adj)
            loss_train += F.l1_loss(output[0][0], y[i,0]) + F.l1_loss(output[1][0], y[i,1]) + F.l1_loss(output[2][0], y[i,2])
        loss += loss_train/i
        loss_train.backward()
        optimizer.step()
    loss = loss/step
    print( "Epoch {}: the train loss = {:.4f}".format(epoch+1, loss))

Epoch 1: the train loss = 1.2013
Epoch 2: the train loss = 0.7811
Epoch 3: the train loss = 0.5748
Epoch 4: the train loss = 0.4112
Epoch 5: the train loss = 0.3108
Epoch 6: the train loss = 0.2734
Epoch 7: the train loss = 0.2623
Epoch 8: the train loss = 0.2572
Epoch 9: the train loss = 0.2530
Epoch 10: the train loss = 0.2492
Epoch 11: the train loss = 0.2455
Epoch 12: the train loss = 0.2420
Epoch 13: the train loss = 0.2386
Epoch 14: the train loss = 0.2334
Epoch 15: the train loss = 0.2224
Epoch 16: the train loss = 0.2125
Epoch 17: the train loss = 0.2049
Epoch 18: the train loss = 0.1994
Epoch 19: the train loss = 0.1955
Epoch 20: the train loss = 0.1927
Epoch 21: the train loss = 0.1907
Epoch 22: the train loss = 0.1891
Epoch 23: the train loss = 0.1879
Epoch 24: the train loss = 0.1869
Epoch 25: the train loss = 0.1861
Epoch 26: the train loss = 0.1854
Epoch 27: the train loss = 0.1849
Epoch 28: the train loss = 0.1844
Epoch 29: the train loss = 0.1840
Epoch 30: the train los

## MAKING PREDICTIONS

In [36]:
predictions = []

# Switching to eval mode
model.eval()

with torch.no_grad():
    for i in range(X_test.size(0)):
        p = model(X_test[i], adj)
        predictions.append(torch.cat([p[0], p[1], p[2]], dim=-1).tolist())

predictions = np.array(predictions)

In [37]:
predictions

array([[0.42028102, 0.3932184 , 0.47370422],
       [0.40622798, 0.37893862, 0.46017754],
       [0.38097081, 0.35303462, 0.43536523],
       ...,
       [0.38946995, 0.36345339, 0.44727695],
       [0.3567906 , 0.3313185 , 0.41806462],
       [0.31992024, 0.29462922, 0.38419926]])

In [38]:
inversed_predictions = mmScaler_load.inverse_transform(predictions)
inversed_predictions

array([[31210.96186208, 11449.35207468, 47998.2372924 ],
       [30601.53398341, 11216.96048622, 47281.10754224],
       [29506.22533032, 10795.39443312, 45965.65919427],
       ...,
       [29874.80136416, 10964.95129372, 46597.17040201],
       [28457.62067231, 10441.98267574, 45048.45106542],
       [26858.6915587 ,  9844.89513351, 43253.04655238]])

In [39]:
inversed_Y_test = mmScaler_load.inverse_transform(Y_test)
inversed_Y_test

array([[31202.114, 11076.755, 43860.518],
       [29720.587, 10566.596, 42079.133],
       [28248.171, 10005.343, 40608.049],
       ...,
       [29583.295, 11166.77 , 46582.286],
       [28458.691, 10838.55 , 45132.982],
       [27428.213, 10592.674, 43501.529]])

In [40]:
MAPE1 = mape(inversed_Y_test[:, 0].reshape(-1), inversed_predictions[:, 0].reshape(-1))
MAPE2 = mape(inversed_Y_test[:, 1].reshape(-1), inversed_predictions[:, 1].reshape(-1))
MAPE3 = mape(inversed_Y_test[:, 2].reshape(-1), inversed_predictions[:, 2].reshape(-1))

print( "The test mape is {:.3f}, {:.3f}, {:.3f}".format(MAPE1, MAPE2, MAPE3))

The test mape is 4.340, 4.865, 4.431


#### AVERAGE MAPE

In [41]:
meanMAPE = np.mean([MAPE1, MAPE2, MAPE3])
meanMAPE

4.545584585521166

#### WEIGHTED MAPE

In [42]:
# Scaled MAPE with max values
weightMax = pd.Series(Load_DS.max() / sum(Load_DS.max())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the max load of a region: ", weightedMAPE)

0    0.366928
1    0.138851
2    0.494221
dtype: float64


0    4.340283
1    4.865299
2    4.431171
dtype: float64


Weighted using the max load of a region:  4.4581011460594056


In [43]:
# Scaled MAPE with average values
weightMax = pd.Series(Load_DS.mean() / sum(Load_DS.mean())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the mean load of a region: ", weightedMAPE)

0    0.345855
1    0.125173
2    0.528972
dtype: float64


0    4.340283
1    4.865299
2    4.431171
dtype: float64


Weighted using the mean load of a region:  4.45407817154547
